In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from itertools import product

import lightgbm as lgb
from tqdm import tqdm_notebook

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [2]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df


In [3]:
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [13]:
print(sales_train.head())
print(test.head())
print(items.head())

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  

In [22]:
test_temp = test.copy()
test_temp['date_block_num'] = 34
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()
test_temp = pd.merge(test_temp, item_category_mapping, how='left', on='item_id')
test_temp = test_temp.drop('ID',axis = 1)
sales_train = sales_train.drop('date', axis = 1)

In [25]:
test_temp
sales_train_full = pd.concat([sales_train,test_temp], ignore_index = True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [26]:
sales_train_full

,date_block_num,item_category_id,item_cnt_day,item_id,item_price,shop_id
0,0,NaN,1.0,22154,999.00,59
1,0,NaN,1.0,2552,899.00,25
2,0,NaN,-1.0,2552,899.00,25
3,0,NaN,1.0,2554,1709.05,25
4,0,NaN,1.0,2555,1099.00,25
...,...,...,...,...,...,...
3150044,34,55.0,NaN,18454,NaN,45
3150045,34,64.0,NaN,16188,NaN,45
3150046,34,55.0,NaN,15757,NaN,45
3150047,34,40.0,NaN,19648,NaN,45


In [27]:
sales = sales_train_full

# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
#all_data2 = all_data.copy()
del grid, gb 
gc.collect();

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


In [28]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [29]:
all_data

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,...,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,54,10297,12,4.0,8198.0,23.0,3.0,42.0,10055.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,54,10296,12,3.0,8198.0,17.0,0.0,24.0,10055.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38
2,54,10298,12,14.0,8198.0,182.0,21.0,369.0,10055.0,119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
3,54,10300,12,3.0,8198.0,26.0,1.0,54.0,10055.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
4,54,10284,12,1.0,8198.0,3.0,0.0,4.0,10055.0,0.0,...,0.0,3.0,7827.0,0.0,10.0,7792.0,0.0,0.0,0.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639289,45,18454,34,0.0,0.0,0.0,1.0,2.0,702.0,0.0,...,0.0,12.0,675.0,0.0,19.0,622.0,0.0,0.0,0.0,55
6639290,45,16188,34,0.0,0.0,0.0,0.0,1.0,702.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64
6639291,45,15757,34,0.0,0.0,0.0,0.0,5.0,702.0,0.0,...,0.0,4.0,675.0,0.0,8.0,622.0,0.0,9.0,1251.0,55
6639292,45,19648,34,0.0,0.0,0.0,0.0,2.0,702.0,0.0,...,0.0,2.0,675.0,0.0,4.0,622.0,0.0,0.0,0.0,40


In [31]:
dates = all_data['date_block_num']

last_block = dates.max()
#print(last_block)

X_train = all_data.loc[dates < last_block - 1].drop(to_drop_cols, axis=1)
X_cv = all_data.loc[dates ==  last_block - 1].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block - 1, 'target'].values
y_cv = all_data.loc[dates ==  last_block - 1, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

# print(X_train_part.shape)
# print(y_train_part.shape)

# X_train.to_pickle("X_train.pkl")
# X_train_part.to_pickle("X_train_part.pkl")
# X_test.to_pickle("X_test.pkl")

# np.save("y_train.npy", y_train)
# np.save("y_train_part.npy", y_train_part)
# np.save("y_test.npy", y_test)

In [32]:
def lgbm_train(X_train,y_train,X_test,y_test,num_iter):
    lgb_params = {
                   'feature_fraction': 0.75,
                   'metric': 'rmse',
                   'nthread':1, 
                   'min_data_in_leaf': 2**7, 
                   'bagging_fraction': 0.75, 
                   'learning_rate': 0.03, 
                   'objective': 'mse', 
                   'bagging_seed': 2**7, 
                   'num_leaves': 2**7,
                   'bagging_freq':1,
                   'verbose':0 
                  }

    evals_result = {}
    
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_valid = lgb.Dataset(X_test, label=y_test)
    model = lgb.train(lgb_params, lgb_train, num_iter,
                      valid_sets=[lgb_train, lgb_valid],
                      evals_result=evals_result,
                      verbose_eval=10)
    return model,evals_result


In [33]:
y_train = y_train.clip(0,40)

print(X_train_part.shape)
print(y_train_part.shape)
model,evals_result  = lgbm_train(X_train, y_train, X_cv, y_cv, 200)

(6186922, 21)
(6186922,)
[10]	training's rmse: 1.28027	valid_1's rmse: 5.27475
[20]	training's rmse: 1.17893	valid_1's rmse: 5.22955
[30]	training's rmse: 1.11709	valid_1's rmse: 5.2001
[40]	training's rmse: 1.07662	valid_1's rmse: 5.18031
[50]	training's rmse: 1.05003	valid_1's rmse: 5.16596
[60]	training's rmse: 1.03286	valid_1's rmse: 5.15535
[70]	training's rmse: 1.01999	valid_1's rmse: 5.14819
[80]	training's rmse: 1.01057	valid_1's rmse: 5.14247
[90]	training's rmse: 1.00314	valid_1's rmse: 5.13792
[100]	training's rmse: 0.996975	valid_1's rmse: 5.13465
[110]	training's rmse: 0.992325	valid_1's rmse: 5.13214
[120]	training's rmse: 0.987735	valid_1's rmse: 5.1306
[130]	training's rmse: 0.98342	valid_1's rmse: 5.12956
[140]	training's rmse: 0.979181	valid_1's rmse: 5.1278
[150]	training's rmse: 0.975536	valid_1's rmse: 5.12724
[160]	training's rmse: 0.972245	valid_1's rmse: 5.12676
[170]	training's rmse: 0.969744	valid_1's rmse: 5.12598
[180]	training's rmse: 0.967444	valid_1's rms

In [ ]:
model.save_model('lgbm_model.txt')

In [ ]:
model = lgb.Booster(model_file='/kaggle/input/predict-future-sales/lgbm_model.txt')

In [43]:
#linear regression in preparation for ensembling
lr = LinearRegression()
y_train = y_train.clip(0,40)

lr.fit(X_train.values, y_train)
# print(X_test.shape)
# print(y_test.shape)
pred_lr = lr.predict(X_cv)

print('Test RMSE for linreg is %f' % np.sqrt(mean_squared_error(y_cv, pred_lr)))

Test RMSE for linreg is 1.042512


In [44]:
pred_lgb = model.predict(X_cv)
pred_lr = pred_lr.clip(0,40)
pred_lgb = pred_lgb.clip(0,40)
X_cv_level2 = np.c_[pred_lr, pred_lgb]
np.save("X_cv_level2.npy", X_cv_level2)

In [ ]:

dates_train = dates[dates <  last_block - 1]
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

count = 0
# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''
    X_train_cur = X_train[dates_train.isin(range(12,cur_block_num))]
    y_train_cur = y_train[dates_train.isin(range(12,cur_block_num))]
    X_test_cur = X_train[dates_train.isin([cur_block_num])]
    
    lr.fit(X_train_cur.values, y_train_cur)
    pred_lr = lr.predict(X_test_cur.values)
    
    model_temp,_ = lgbm_train(X_train_cur, y_train_cur, X_train_cur, y_train_cur, 200)
    pred_lgb = model_temp.predict(X_test_cur)
    
    for i in range(len(pred_lr)):
        X_train_level2[count+i][0]=pred_lr[i]
        X_train_level2[count+i][1]=pred_lgb[i]
    count+=len(pred_lr)
np.save("X_train_full_level2.npy", X_train_level2)
np.save("y_train_full_level2.npy", y_train_level2)

In [46]:
X_train_level2 = np.load("X_train_full_level2.npy")
y_train_level2 = np.load("y_train_full_level2.npy")

lr.fit(X_train_level2, y_train_level2)

test_preds = lr.predict(X_cv_level2)
rmse_test_stacking = np.sqrt(mean_squared_error(y_cv, test_preds))
rmse_test_stacking

1.0064872972254926

In [ ]:
pred_lgb_test = model.predict(X_test)
pred_ens_test = best_alpha*pred_lgb_test + (1-best_alpha)*pred_lr


In [ ]:
to_drop_cols = ['ID','date_block_num']
X_test_final =  test_data.drop(to_drop_cols, axis=1)
pred_lgb_final = model_full.predict(X_test_final)
# pred_lr_final = lr_full.predict(X_test_final)
#pred_ens = best_alpha*pred_lgb_final + (1-best_alpha)*pred_lr_final
#print(pred_lgb)

merged = test.copy()
merged['item_cnt_month'] = pred_lgb_final
merged['item_cnt_month']=merged['item_cnt_month'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id'],axis=1)
merged['ID']=merged['ID'].astype('int')
merged.to_csv("lightgbm_clipto40.csv",index=False)
print(merged.head)